# Creating Binary Treatment and Event Count Variables for Brazil Firm-Disaster Data

## Overview
This code processes the Brazil firm-disaster distances dataset (`brazil_est_shock_distances.csv`) to create a binary treatment variable and an event count variable. It merges the known disaster data with the subsidiary level (cnpj_cei) data. The output is one dataset per year, with each row representing a unique establishment-year combination.

- `treated`: binary =1 if the establishment (unique id cnpj_cei) is inside the subtype-specific radius of at least one disaster that year; 0 otherwise. (`flood`,`storm`, `landslide`: 25km; `earthquake`: 100km).
- `num_treatments`: integer count of how many distinct disasters hit that establishment within its radius in that year.

In [50]:
import pandas as pd
import numpy as np

In [51]:
df = pd.read_csv('./geocoded_data/brazil_est_shock_distances.csv')
df

,year,disaster_id,est_id,lat_disaster,lng_disaster,lat_est,lng_est,disaster_type,distance_km,total_deaths,total_damage
0,2013,3036_2013,2.916265e+12,-22.587911,-43.326527,-10.033735,-62.977755,flood,2514.201261,4.0,2000.0
1,2013,3036_2013,8.461554e+13,-22.587911,-43.326527,-9.908812,-63.035547,flood,2527.515565,4.0,2000.0
2,2013,3036_2013,6.900697e+12,-22.587911,-43.326527,-9.903959,-63.034621,flood,2527.753442,4.0,2000.0
3,2013,3036_2013,3.780605e+12,-22.587911,-43.326527,-9.898254,-63.034756,flood,2528.141185,4.0,2000.0
4,2013,3036_2013,4.082624e+12,-22.587911,-43.326527,-9.898254,-63.034756,flood,2528.141185,4.0,2000.0
...,...,...,...,...,...,...,...,...,...,...,...
8999213,2015,44701_2015,4.750841e+13,-29.737839,-53.316837,-15.708171,-47.914884,flood,1654.775691,NaN,NaN
8999214,2015,44701_2015,4.750841e+13,-29.737839,-53.316837,-15.809553,-47.880150,flood,1645.286293,NaN,NaN
8999215,2015,44701_2015,4.750841e+13,-29.737839,-53.316837,-15.820243,-47.904509,flood,1643.322623,NaN,NaN
8999216,2015,44701_2015,4.750841e+13,-29.737839,-53.316837,-15.745426,-47.922018,flood,1650.605674,NaN,NaN


In [52]:
df["disaster_type"].value_counts()

disaster_type
flood         8433493
storm          353832
landslide      179163
earthquake      32730
Name: count, dtype: int64

In [53]:
disaster_subtype_radii = {
    'flood': 25.0,
    'storm': 25.0,
    'landslide': 25.0,
    'earthquake': 100.0
}

In [54]:
# 1. Assign 'within_radius' based on disaster type
df['within_radius'] = df.apply(
    lambda row: int(row['distance_km'] <= disaster_subtype_radii.get(row['disaster_type'])),
    axis=1
)
df.head()

,year,disaster_id,est_id,lat_disaster,lng_disaster,lat_est,lng_est,disaster_type,distance_km,total_deaths,total_damage,within_radius
0,2013,3036_2013,2.916265e+12,-22.587911,-43.326527,-10.033735,-62.977755,flood,2514.201261,4.0,2000.0,0
1,2013,3036_2013,8.461554e+13,-22.587911,-43.326527,-9.908812,-63.035547,flood,2527.515565,4.0,2000.0,0
2,2013,3036_2013,6.900697e+12,-22.587911,-43.326527,-9.903959,-63.034621,flood,2527.753442,4.0,2000.0,0
3,2013,3036_2013,3.780605e+12,-22.587911,-43.326527,-9.898254,-63.034756,flood,2528.141185,4.0,2000.0,0
4,2013,3036_2013,4.082624e+12,-22.587911,-43.326527,-9.898254,-63.034756,flood,2528.141185,4.0,2000.0,0


In [55]:
# 2. Create a 'treated' flag for each establishment-year combination
#    indicating if the establishment was within the radius of any disaster in that year
treated_flags = (
    df.groupby(['est_id', 'year'])[['within_radius', 'total_deaths', 'total_damage']]
    .agg(
        within_radius=('within_radius', 'max'),
        total_deaths=('total_deaths', 'sum'),
        total_damage=('total_damage', 'sum')
    )
    .reset_index()
    .rename(columns={'within_radius': 'treated'})
)
treated_flags

,est_id,year,treated,total_deaths,total_damage
0,7.870000e+02,2006,0,52.0,0.0
1,5.844010e+05,2011,0,3050.0,3012000.0
2,1.180001e+08,2003,0,2511.0,3030000.0
3,1.180001e+08,2004,0,43.0,700370.0
4,1.180001e+08,2006,0,52.0,0.0
...,...,...,...,...,...
549070,9.875036e+13,2013,0,307.0,1271500.0
549071,9.875036e+13,2014,0,162.0,400000.0
549072,9.875036e+13,2015,0,37.0,6000.0
549073,9.875036e+13,2016,0,201.0,1300000.0


In [56]:
treated_flags['treated'].value_counts()

treated
0    520064
1     29011
Name: count, dtype: int64

In [57]:
# 3. Count the number of disasters per establishment-year combination
treatment_counts = (
    df[df['within_radius'] == 1]
    .groupby(['est_id', 'year'])['disaster_id']
    .nunique()
    .reset_index()
    .rename(columns={'disaster_id': 'treatment_count'})
)
treatment_counts

,est_id,year,treatment_count
0,1.392000e+09,2007,1
1,1.392000e+09,2010,1
2,3.516000e+09,2016,1
3,5.103000e+09,2010,1
4,5.275000e+09,2010,1
...,...,...,...
29006,9.852225e+13,2005,2
29007,9.852225e+13,2003,3
29008,9.852225e+13,2009,3
29009,9.852225e+13,2010,1


In [58]:
assert treated_flags['treated'].sum() == treatment_counts.shape[0], "The number of treated establishments should match the number of establishments with treatment counts."


In [59]:
# 4. Merge the treated flags with the treatment counts
est_year_treatments = treated_flags.merge(
    treatment_counts,
    on=['est_id', 'year'],
    how='left'
)
est_year_treatments['treatment_count'] = est_year_treatments['treatment_count'].fillna(0).astype(int)
est_year_treatments

,est_id,year,treated,total_deaths,total_damage,treatment_count
0,7.870000e+02,2006,0,52.0,0.0,0
1,5.844010e+05,2011,0,3050.0,3012000.0,0
2,1.180001e+08,2003,0,2511.0,3030000.0,0
3,1.180001e+08,2004,0,43.0,700370.0,0
4,1.180001e+08,2006,0,52.0,0.0,0
...,...,...,...,...,...,...
549070,9.875036e+13,2013,0,307.0,1271500.0,0
549071,9.875036e+13,2014,0,162.0,400000.0,0
549072,9.875036e+13,2015,0,37.0,6000.0,0
549073,9.875036e+13,2016,0,201.0,1300000.0,0


In [60]:
assert est_year_treatments['treated'].sum() == treated_flags['treated'].sum(), "The total number of treated establishments should match."
assert est_year_treatments['treatment_count'].sum() == treatment_counts['treatment_count'].sum(), "The total treatment counts should match."

In [61]:
# 5. Merge the binary treatment flag and treatment counts with the ESTB data
# Operate on each estb file separately because they are too big to merge all at once
import os
import gc
estb_path = '/Users/koacow/BOSTON UNIVERSITY Dropbox/Ngoc Duy Khoa Cao/Climate Risk and Labor Market/RAIS data/firms and cities/ESTB'
outpath = '/Users/koacow/BOSTON UNIVERSITY Dropbox/Ngoc Duy Khoa Cao/Climate Risk and Labor Market/RAIS data/firms and cities/ESTB/treated'

for fname in sorted(os.listdir(estb_path)):
    if fname.startswith("estb2") and fname.endswith(".csv"):
        year = int(fname[4:8])  # extract year from filename like estb2004.csv
        full_path = os.path.join(estb_path, fname)
        df_year = pd.read_csv(full_path, encoding='iso-8859-1')
        df_year['year'] = year
        df_year = df_year.merge(est_year_treatments, left_on=['cnpj_cei', 'year'], right_on=['est_id', 'year'], how='left')
        df_year = df_year.drop(columns=['est_id'])
        df_year['treated'] = df_year['treated'].fillna(0).astype(int)
        df_year['treatment_count'] = df_year['treatment_count'].fillna(0).astype(int)
        df_year["cnpj_cei"] = df_year["cnpj_cei"].astype(str)
        df_year["cnpj_cei"] = df_year["cnpj_cei"].str.replace('.0', '', regex=False)
        df_year["cnpj_cei"] = df_year["cnpj_cei"].str.zfill(14)
        df_year.to_csv(os.path.join(outpath, f'estb_{year}_treated.csv'), index=False)
        del df_year
        gc.collect()

/var/folders/zx/x6hb1_p95p33hh3235qrnlqm0000gn/T/ipykernel_66755/115749416.py:12: DtypeWarning: Columns (0,1,6,16) have mixed types. Specify dtype option on import or set low_memory=False.
  df_year = pd.read_csv(full_path, encoding='iso-8859-1')
/var/folders/zx/x6hb1_p95p33hh3235qrnlqm0000gn/T/ipykernel_66755/115749416.py:12: DtypeWarning: Columns (0,5,6,16) have mixed types. Specify dtype option on import or set low_memory=False.
  df_year = pd.read_csv(full_path, encoding='iso-8859-1')
/var/folders/zx/x6hb1_p95p33hh3235qrnlqm0000gn/T/ipykernel_66755/115749416.py:12: DtypeWarning: Columns (0,1,5,6,16) have mixed types. Specify dtype option on import or set low_memory=False.
  df_year = pd.read_csv(full_path, encoding='iso-8859-1')
/var/folders/zx/x6hb1_p95p33hh3235qrnlqm0000gn/T/ipykernel_66755/115749416.py:12: DtypeWarning: Columns (0,5,6,16) have mixed types. Specify dtype option on import or set low_memory=False.
  df_year = pd.read_csv(full_path, encoding='iso-8859-1')
/var/folde

In [62]:
total_treated = 0
total_treatment_counts = 0
nan_id_count = 0
for fname in sorted(os.listdir(outpath)):
    if fname.startswith("estb_2") and fname.endswith(".csv"):
        full_path = os.path.join(outpath, fname)
        treated_df = pd.read_csv(full_path, encoding='iso-8859-1')
        total_treated += treated_df['treated'].sum()
        total_treatment_counts += treated_df['treatment_count'].sum()
        nan_id_count += treated_df["cnpj_cei"].isna().sum()

assert total_treated == est_year_treatments['treated'].sum(), "Total treated establishments should match across all years."
assert total_treatment_counts == est_year_treatments['treatment_count'].sum(), "Total treatment counts should match across all years."
print(f"Total treated establishments: {total_treated}")
print(f"Total treatment counts: {total_treatment_counts}")
print(f"Total NaN CNPJ/CEI IDs across all years: {nan_id_count}")

/var/folders/zx/x6hb1_p95p33hh3235qrnlqm0000gn/T/ipykernel_66755/1105905189.py:7: DtypeWarning: Columns (0,1,3,6,16) have mixed types. Specify dtype option on import or set low_memory=False.
  treated_df = pd.read_csv(full_path, encoding='iso-8859-1')
/var/folders/zx/x6hb1_p95p33hh3235qrnlqm0000gn/T/ipykernel_66755/1105905189.py:7: DtypeWarning: Columns (0,3,5,6,16) have mixed types. Specify dtype option on import or set low_memory=False.
  treated_df = pd.read_csv(full_path, encoding='iso-8859-1')
/var/folders/zx/x6hb1_p95p33hh3235qrnlqm0000gn/T/ipykernel_66755/1105905189.py:7: DtypeWarning: Columns (0,1,3,5,6,16) have mixed types. Specify dtype option on import or set low_memory=False.
  treated_df = pd.read_csv(full_path, encoding='iso-8859-1')
/var/folders/zx/x6hb1_p95p33hh3235qrnlqm0000gn/T/ipykernel_66755/1105905189.py:7: DtypeWarning: Columns (0,3,5,6,16) have mixed types. Specify dtype option on import or set low_memory=False.
  treated_df = pd.read_csv(full_path, encoding='iso

Total treated establishments: 29011
Total treatment counts: 46710
Total NaN CNPJ/CEI IDs across all years: 0
